In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
import random
import gensim as gs
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer, TFBertMainLayer, BertConfig

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU not found')
print('found GPU at {}'.format(device_name))

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')

In [ ]:
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
bert_model = TFBertModel.from_pretrained("bert-base-cased")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [ ]:
MAX_TOKENS = 125

In [ ]:
articles = pd.read_csv('../../dataset/data_articles_test.csv', delimiter=',', keep_default_na=False)

tables = pd.read_csv('../../dataset/distinct_tables_allsignals.csv', delimiter=',', keep_default_na=False)
tables = tables.drop(tables[tables.table_page_title == ""].index)

In [ ]:
def triplet_loss(y_true, y_pred, alpha = 0.5):
     
    anchor = y_pred[:,0:768]
    positive = y_pred[:,768:1536]
    negative = y_pred[:,1536:2304]
        
    # distance between the anchor and the positive
    pos_dist = tf.keras.layers.Dot(axes=1,normalize=True)([anchor, positive])
    
    # distance between the anchor and the negative
    neg_dist = tf.keras.layers.Dot(axes=1,normalize=True)([anchor, negative])
    
    # compute loss
    basic_loss = (1 - pos_dist) - (1 - neg_dist) + alpha
    loss = tf.keras.backend.maximum(basic_loss,0.0)
 
    return loss

In [ ]:
loaded_model = tf.keras.models.load_model('bert_encoder_model2', custom_objects={'triplet_loss': triplet_loss})

In [ ]:
layer_name = 'bert'
intermediate_layer_model = tf.keras.Model(inputs=[loaded_model.input[0],loaded_model.input[1],loaded_model.input[2]],
                                 outputs=loaded_model.get_layer(layer_name).output)

## encode articles using encoder model

In [ ]:
articles_ids = []
input_ids_article_title = []
input_masks_article_title = []
input_token_id_article_title = []

for i, row in tqdm(articles.iterrows()):
    
#     article_catch_all = str(row['page_title'])+" "+ str(row['meta_description'])
    article_catch_all = str(row['page_title']) +" "+ str(row['meta_description'])+" "+str(row['keywords'])
      
    #fast text embedding
    return_tokenizer1 = bert_tokenizer.encode_plus(
      article_catch_all,
      max_length=MAX_TOKENS,
      add_special_tokens=True,
      return_token_type_ids=True,
      pad_to_max_length=True,
      return_attention_mask=True,
    )
    
    #save list
    articles_ids.append(row['page_id'])
    
    input_ids_article_title.append(return_tokenizer1['input_ids'])
    input_masks_article_title.append(return_tokenizer1['attention_mask'])
    input_token_id_article_title.append(return_tokenizer1['token_type_ids']) 

In [ ]:
input_ids_article_title = np.array(input_ids_article_title)
input_masks_article_title = np.array(input_masks_article_title)
input_token_id_article_title = np.array(input_token_id_article_title)

In [ ]:
input_token_id_article_title.shape

In [ ]:
articles_vector = np.asarray(intermediate_layer_model.predict([input_ids_article_title,input_masks_article_title,input_token_id_article_title])[1])

In [ ]:
articles_vector.shape

In [ ]:
tables_ids = []
input_ids_tables_title = []
input_masks_tables_title = []
input_token_id_tables_title = []

for i, row in tqdm(tables.iterrows()):
    
#     table_catch_all = str(row['table_page_title'])+" "+str(row['table_page_summary'])
    table_catch_all = str(row['table_page_title'])+" "+str(row['table_page_summary'])+" "+str(row['table_page_keywords'])
    
    #fast text embedding
    return_tokenizer1 = bert_tokenizer.encode_plus(
      table_catch_all,
      max_length=MAX_TOKENS,
      add_special_tokens=True,
      return_token_type_ids=True,
      pad_to_max_length=True,
      return_attention_mask=True,
    )
    
    tables_ids.append(row['table_id'])
    
    input_ids_tables_title.append(return_tokenizer1['input_ids'])
    input_masks_tables_title.append(return_tokenizer1['attention_mask'])
    input_token_id_tables_title.append(return_tokenizer1['token_type_ids']) 

In [ ]:
input_ids_tables_title = np.array(input_ids_tables_title)
input_masks_tables_title = np.array(input_masks_tables_title)
input_token_id_tables_title = np.array(input_token_id_tables_title)

In [ ]:
input_ids_tables_title.shape

In [ ]:
tables_vector = []

first = 0

for i in tqdm(range(100,(len(input_ids_tables_title)+100),100)):
    
    tables_vector.append(np.asarray(intermediate_layer_model.predict([input_ids_tables_title[first:i],input_masks_tables_title[first:i],input_token_id_tables_title[first:i]])[1]))
    
    first = first + 100

In [ ]:
tables_vector = np.array(tables_vector)

In [ ]:
tables_vector.shape

In [ ]:
# tables_vector_final = tables_vector.reshape(85900,768)
tables_vector_final = tables_vector.reshape(-1,768)

In [ ]:
tables_vector_final.shape

## test the model on final task

In [ ]:
def getAccuracy(idRankedTables, idQueryGoal):

    accuracy = 0

    for idTable in idRankedTables:
        
        if idTable[0] == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [ ]:
def saveAccuracy(k,accuracy):
    
    if k == 1:
            
        AverageTop1.append(accuracy)
    
    if k == 5:
            
        AverageTop5.append(accuracy)
        
    if k == 10:
            
        AverageTop10.append(accuracy)
        
    if k == 20:
            
        AverageTop20.append(accuracy)
    
    if k == 50:
            
        AverageTop50.append(accuracy)
    
    if k == 100:
            
        AverageTop100.append(accuracy)

In [ ]:
AverageTop1 = []
AverageTop5 = []
AverageTop10 = []
AverageTop20 = []
AverageTop50 = []
AverageTop100 = []

topK = [1,5,10,20,50,100]

for i in tqdm(range(len(articles_vector))):
    
    idQueryGoal = articles_ids[i]
    
    distance_vector = pairwise_distances(articles_vector[i].reshape(1,768), tables_vector_final, metric='cosine')
    
    #creating the dataframe
    all_tables_score = []
    
    for j in range(len(tables_ids)):
        
        table_id = tables_ids[j]
        table_score = distance_vector[0][j]
        
        new_row = {"table_id": table_id,"table_score": table_score}
        
        all_tables_score.append(new_row)
        
    df_all_tables_scores = pd.DataFrame(all_tables_score)
    df_tables_sorting = df_all_tables_scores.sort_values('table_score')
    
    #compute the accuracy
    for accuracyK in topK:
        
        selected_top = df_tables_sorting.head(accuracyK)
        
        min_score = selected_top['table_score'].max()
        draw_tables_socres = df_tables_sorting[df_tables_sorting['table_score'] <= min_score]
        final_ranked_tables = draw_tables_socres.iloc[:,0:1].values
        
        accuracy_value = getAccuracy(final_ranked_tables,idQueryGoal)
        
        #save the accuracy on the list
        saveAccuracy(accuracyK,accuracy_value)

In [ ]:
print("TOP@1 = "+ str(round(np.mean(AverageTop1),4)))
print("TOP@5 = "+ str(round(np.mean(AverageTop5),4)))
print("TOP@10 = "+ str(round(np.mean(AverageTop10),4)))
print("TOP@20 = "+ str(round(np.mean(AverageTop20),4)))
print("TOP@50 = "+ str(round(np.mean(AverageTop50),4)))
print("TOP@100 = "+ str(round(np.mean(AverageTop100),4)))

In [ ]:
selected_top

In [ ]:
idQueryGoal